Notebook a utilizar el Google Colab para el entrenamiento de YOLO

### Activar GPU

In [ ]:
!nvidia-smi

# Deberías ver algo como:
# | NVIDIA-SMI 525.xx.xx    Driver Version: 525.xx.xx    CUDA Version: 12.x |
# |   0  Tesla T4           ...

import torch
print(f"\n✅ CUDA disponible: {torch.cuda.is_available()}")
print(f"✅ GPU detectada: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else '❌ NO GPU'}")

# Si NO ves la GPU:
# 1. Ve a: Entorno de ejecución > Cambiar tipo de entorno de ejecución
# 2. Selecciona: Acelerador por hardware > GPU
# 3. Guarda y ejecuta esta celda de nuevo

### Instalar Ultralytics para utilizar YOLO

In [ ]:
print("📦 Instalando Ultralytics...")
!pip install ultralytics -q

# Verificar instalación
from ultralytics import YOLO
import ultralytics
print(f"✅ Ultralytics instalado: versión {ultralytics.__version__}")

### Habilitar Google Drive para acceder al dataset y almacenar los resultados del entrenamiento

In [ ]:
from google.colab import drive
print("🔗 Montando Google Drive...")
drive.mount('/content/drive')

# Te pedirá permisos:
# 1. Clic en el enlace
# 2. Selecciona tu cuenta de Google
# 3. Permite el acceso
# 4. Copia el código y pégalo

print("✅ Drive montado correctamente")

# Crear carpeta del proyecto
import os
project_path = '/content/drive/MyDrive/YOLO11_Dron'
os.makedirs(project_path, exist_ok=True)
print(f"📁 Carpeta del proyecto: {project_path}")

### Verificar la configuración inicial de trabajo:  
GPU, Google Drive y Ultralytics

In [ ]:
print("🔍 Verificando configuración...\n")

# 1. GPU
if torch.cuda.is_available():
    print("✅ GPU: OK")
    print(f"   └─ {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU: NO DETECTADA")
    print("   └─ Ve a: Entorno de ejecución > Cambiar tipo > GPU")

# 2. Drive
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive: OK")
else:
    print("❌ Google Drive: NO MONTADO")

# 3. Ultralytics
try:
    from ultralytics import YOLO
    print(f"✅ Ultralytics: OK (v{ultralytics.__version__})")
except:
    print("❌ Ultralytics: NO INSTALADO")

print("\n🎉 Si todo está en ✅, ¡estás listo para entrenar!")

### Verificar yaml

In [ ]:
from ultralytics import YOLO
import yaml
import os

# Cargar modelo
model = YOLO('yolo11n.pt')

# Ruta al dataset AirSim en Drive
AirSim_dataset_path = '/content/drive/MyDrive/YOLO11_Dron/datasets/AirSim_unificado'
AirSim_yaml_path = os.path.join(AirSim_dataset_path, 'merged.yaml')

print("🔍 Verificando dataset AirSim...")
print(f"\n📁 Ruta: {AirSim_dataset_path}")

# Verificar estructura
estructura_esperada = {
    'train/images': 'Imágenes de entrenamiento',
    'test/images': 'Imágenes de test',
    'valid/images': 'Imágenes de validación',
    'train/labels': 'Etiquetas de entrenamiento',
    'test/labels': 'Etiquetas de test',
    'valid/labels': 'Etiquetas de validación',
    'merged.yaml': 'Archivo de configuración'
}

print("\n📊 Verificando estructura:")
todo_ok = True

for ruta, descripcion in estructura_esperada.items():
    ruta_completa = os.path.join(AirSim_dataset_path, ruta)
    existe = os.path.exists(ruta_completa)

    if existe:
        if os.path.isfile(ruta_completa):
            print(f"   ✅ {ruta}")
        else:
            num_archivos = len([f for f in os.listdir(ruta_completa)
                               if os.path.isfile(os.path.join(ruta_completa, f))])
            print(f"   ✅ {ruta} ({num_archivos} archivos)")
    else:
        print(f"   ❌ {ruta} - NO ENCONTRADO")
        todo_ok = False

if not todo_ok:
    print("\n⚠️ ADVERTENCIA: Faltan archivos en la estructura")
    print("   Verifica que hayas subido todo correctamente a Drive")

# Verificar archivo YAML
print("\n📋 Contenido de merged.yaml:")
if os.path.exists(AirSim_yaml_path):
    with open(AirSim_yaml_path, 'r') as f:
        AirSim_config = yaml.safe_load(f)

    print(yaml.dump(AirSim_config, default_flow_style=False, allow_unicode=True))

    # Verificar configuración
    print("🔍 Validando configuración:")

    # Verificar paths
    if 'path' in AirSim_config:
        print(f"   ✅ path: {AirSim_config['path']}")
    else:
        print(f"   ⚠️ 'path' no especificado, usando: {AirSim_dataset_path}")
        AirSim_config['path'] = AirSim_dataset_path

    if 'train' in AirSim_config:
        print(f"   ✅ train: {AirSim_config['train']}")
    else:
        print(f"   ❌ 'train' no especificado")

    if 'val' in AirSim_config:
        print(f"   ✅ val: {AirSim_config['val']}")
    else:
        print(f"   ❌ 'val' no especificado")

    # Verificar clases
    if 'nc' in AirSim_config and 'names' in AirSim_config:
        print(f"   ✅ Clases: {AirSim_config['nc']}")
        print(f"   📝 Nombres: {AirSim_config['names']}")
    else:
        print(f"   ❌ Clases no especificadas correctamente")

else:
    print("   ❌ merged.yaml NO ENCONTRADO")
    print("\n⚠️ CRÍTICO: Necesitas el archivo data.yaml")
    AirSim_config = None

# Ver ejemplos de imágenes y etiquetas
if todo_ok:
    train_imgs = os.path.join(AirSim_dataset_path, 'train/images')
    train_labels = os.path.join(AirSim_dataset_path, 'train/labels')

    imagenes = [f for f in os.listdir(train_imgs) if f.endswith(('.jpg', '.png', '.jpeg'))]
    etiquetas = [f for f in os.listdir(train_labels) if f.endswith('.txt')]

    print(f"\n📊 Estadísticas del dataset:")
    print(f"   Imágenes train: {len(imagenes)}")
    print(f"   Etiquetas train: {len(etiquetas)}")

    val_imgs = os.path.join(AirSim_dataset_path, 'valid/images')
    val_labels = os.path.join(AirSim_dataset_path, 'valid/labels')

    if os.path.exists(val_imgs):
        imagenes_val = [f for f in os.listdir(val_imgs) if f.endswith(('.jpg', '.png', '.jpeg'))]
        etiquetas_val = [f for f in os.listdir(val_labels) if f.endswith('.txt')]
        print(f"   Imágenes val: {len(imagenes_val)}")
        print(f"   Etiquetas val: {len(etiquetas_val)}")

    # Mostrar imagen de ejemplo
    if imagenes:
        from IPython.display import Image, display

        print("\n🖼️ Ejemplo de imagen del dataset:")
        sample_img = os.path.join(train_imgs, imagenes[0])
        display(Image(filename=sample_img, width=600))
        print(f"   {imagenes[0]}")

        # Mostrar etiqueta correspondiente
        sample_label = os.path.join(train_labels, imagenes[0].replace('.jpg', '.txt').replace('.png', '.txt'))
        if os.path.exists(sample_label):
            print(f"\n📝 Contenido de la etiqueta (primeras 5 líneas):")
            with open(sample_label, 'r') as f:
                lines = f.readlines()[:5]
                for line in lines:
                    print(f"   {line.strip()}")


### Entrenamiento del modelo YOLO11n con el dataset creado a partir de roboflow de imagenes de AirSim

In [ ]:

# Entrenamiento completo
results = model.train(
    data=AirSim_yaml_path,  # Se corrigió la ruta de 'merged.yaml' a AirSim_yaml_path
    epochs=100,                      # Entrenamiento completo
    imgsz=640,
    batch=16,                        # Reducir a 8 si da error
    device=0,

    # Guardar en Drive
    project='/content/drive/MyDrive/YOLO11_Dron/runs',
    name='AirSim',

    # Checkpoints frecuentes por si se corta
    save=True,
    save_period=10,                  # Guardar cada 10 épocas

    # Configuración óptima
    patience=10,
    optimizer='AdamW',
    lr0=0.001,

    # Augmentaciones
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,

    verbose=True
)

print("\n🎉 ¡ENTRENAMIENTO COMPLETADO!")

### **Reanudar** entrenamiento

In [ ]:
from ultralytics import YOLO

print("🔄 Reanudando entrenamiento...")

# Cargar último checkpoint
model = YOLO('/content/drive/MyDrive/YOLO11_Dron/runs/AirSim2/weights/last.pt')

# Continuar entrenamiento
results = model.train(
    resume=True,  # ← IMPORTANTE
    project='/content/drive/MyDrive/YOLO11_Dron/runs',
    name='AirSim'
)

print("✅ Entrenamiento reanudado")

### Métricas del modelo

In [ ]:
from IPython.display import Image, display
import os

results_path = '/content/drive/MyDrive/YOLO11_Dron/runs/AirSim2'

print("📊 Visualizando resultados...\n")

# Mostrar curvas de entrenamiento
if os.path.exists(f'{results_path}/results.png'):
    print("📈 Curvas de entrenamiento:")
    display(Image(filename=f'{results_path}/results.png', width=800))

# Mostrar matriz de confusión
if os.path.exists(f'{results_path}/confusion_matrix.png'):
    print("\n🎯 Matriz de confusión:")
    display(Image(filename=f'{results_path}/confusion_matrix.png', width=600))

# Mostrar ejemplos de predicciones
if os.path.exists(f'{results_path}/val_batch0_pred.jpg'):
    print("\n🖼️ Ejemplos de predicciones:")
    display(Image(filename=f'{results_path}/val_batch0_pred.jpg', width=800))

# Ver métricas
import pandas as pd
if os.path.exists(f'{results_path}/results.csv'):
    df = pd.read_csv(f'{results_path}/results.csv')
    print("\n📊 Métricas finales:")
    print(df[['epoch', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail())

In [ ]:
from ultralytics import YOLO

# Cargar mejor modelo
best_model = YOLO('/content/drive/MyDrive/YOLO11_Dron/runs/AirSim2/weights/best.pt')

# Validar
print("🎯 Validando modelo...")
metrics = best_model.val()

print(f"\n📊 RESULTADOS FINALES:")
print(f"   mAP50: {metrics.box.map50:.4f} ({metrics.box.map50*100:.1f}%)")
print(f"   mAP50-95: {metrics.box.map:.4f} ({metrics.box.map*100:.1f}%)")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall: {metrics.box.mr:.4f}")